We load our points data set (see *data_prep1*) and the DEM (resolution 30x30).

In [1]:
using CSV, DataFrames, GeoArrays

points_a = CSV.read("data/points_a.csv")
points_f = CSV.read("data/points_f.csv")
aster = GeoArrays.read("data/aster_clipped.tif") # from https://gdemdl.aster.jspacesystems.or.jp/index_en.html
bounds = size(aster) # the boundaries (max x and max y) of the DEM

(206, 327, 1)

Next, we map all geographic coordinates onto the grid and store the corresponding grid coordinates. We remove all segments shorter than 4 raster cells (so roughly 120 m).

In [2]:
using Query, Dates

function process_points(df::DataFrame, suffix::String)
    df.index = [indices(aster, [x, y]) for (x,y) in zip(df.lon, df.lat)] # store corresponding raster coordinates
    df.xindex = map(x -> x[1], df.index)
    df.yindex = map(x -> x[2], df.index)

    df.i_xy = [(x, y) for (x, y) in zip(df.xindex, df.yindex)]
    replace!(df.altitude, missing => 0)
    df.altitude = Float64.(df.altitude)
    points_grouped_ = df |> # create df where each row corresponds to one segment, col. i_xy holds an array of waypoints (grid coords)
    @groupby(_.segment) |> 
    @map({segment=key(_), i_xy=vcat(_.i_xy), date=Date(_.datetime[1]), focal=_.focal[1], altitudes=vcat(_.altitude)}) |> 
    DataFrame

    println(suffix, " - number of segments (total): ", nrow(points_grouped_))
    points_grouped = points_grouped_[[abs(x[1][1] - x[end][1]) > 3 || abs(x[1][2] - x[end][2]) > 3 for x in points_grouped_.i_xy], :]
    println(suffix, " - number of segments longer than 4 cells: ", nrow(points_grouped))

    println(suffix, " - number of points (total): ", nrow(df))
    points = df[[in(x, points_grouped.segment) for x in df.segment], :]
    println(suffix, " - number of points (segments longer than 4 cells): ", nrow(points))

    CSV.write("data/points_raster_" * suffix * ".csv", points)
end

process_points(points_a, "a")
process_points(points_f, "f")

a - number of segments (total): 2458
a - number of segments longer than 4 cells: 1163
a - number of points (total): 35085
a - number of points (segments longer than 4 cells): 29415
f - number of segments (total): 731
f - number of segments longer than 4 cells: 620
f - number of points (total): 33310
f - number of points (segments longer than 4 cells): 32367


"data/points_raster_f.csv"